# Ad-Hoc Reporting

Utilize your `sqlite3` and `pandas` knowledge to query your database and generate visualizations! 

After completing your report, answer the 4 analytical questions below. Remember that we expect comprehensive numerical detail! 

In [2]:
import sqlite3

import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# TODO: Set up a connection the "Market.db" database
conn = sqlite3.connect("db/Market.db")

# TODO: Set up the cursor
cursor = conn.cursor()

In [ ]:
#all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())


[('Categories',), ('sqlite_sequence',), ('Customers',), ('Employees',), ('Shippers',), ('Suppliers',), ('Products',), ('Orders',), ('OrderDetails',)]


In [ ]:
# Product table
cursor.execute("SELECT * FROM Products")
print(cursor.description) ##column info


(('ProductID', None, None, None, None, None, None), ('ProductName', None, None, None, None, None, None), ('SupplierID', None, None, None, None, None, None), ('CategoryID', None, None, None, None, None, None), ('Unit', None, None, None, None, None, None), ('Price', None, None, None, None, None, None))


In [ ]:
#  Products - columns
cursor.execute("SELECT * FROM Products")
columns = [desc[0] for desc in cursor.description] 

print("Column names:", columns)

Column names: ['ProductID', 'ProductName', 'SupplierID', 'CategoryID', 'Unit', 'Price']


In [8]:
def run_query(query):
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    return pd.DataFrame(rows, columns=columns)

In [15]:
run_query(" select * from Products")

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18.00
1,2,Chang,1,1,24 - 12 oz bottles,19.00
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35
...,...,...,...,...,...,...
72,73,Röd Kaviar,17,8,24 - 150 g jars,15.00
73,74,Longlife Tofu,4,7,5 kg pkg.,10.00
74,75,Rhönbräu Klosterbier,12,1,24 - 0.5 l bottles,7.75
75,76,Lakkalikööri,23,1,500 ml,18.00


### Question 1 : Total Products Less than €10

How many products in our Products table cost less than 10 EUR?

**Hint**: You can apply an aggregate function to an entire table.

In [19]:
# TODO: Write a SQL query which will answer the question above.
cursor.execute("""
    select  count(*) from Products where price <10
               
""")

rows = cursor.fetchall()
print(rows)

[(11,)]


### Question 2 : Most Frequent Country by Supplier

What is the most common country of origin in our Suppliers table? Sort your output in descending order.

**Hint**: `GROUP BY` can be used to calculate grouped aggregates.

In [21]:
run_query(" select * from suppliers limit 5")

,SupplierID,SupplierName,ContactName,Address,City,PostalCode,Country,Phone
0,1,Exotic Liquid,Charlotte Cooper,49 Gilbert St.,Londona,EC1 4SD,UK,(171) 555-2222
1,2,New Orleans Cajun Delights,Shelley Burke,P.O. Box 78934,New Orleans,70117,USA,(100) 555-4822
2,3,Grandma Kelly's Homestead,Regina Murphy,707 Oxford Rd.,Ann Arbor,48104,USA,(313) 555-5735
3,4,Tokyo Traders,Yoshi Nagase,9-8 Sekimai Musashino-shi,Tokyo,100,Japan,(03) 3555-5011
4,5,Cooperativa de Quesos 'Las Cabras',Antonio del Valle Saavedra,Calle del Rosal 4,Oviedo,33007,Spain,(98) 598 76 54


In [22]:
# TODO: Write a SQL query which will answer the question above.
cursor.execute("""
    select Country,count(Country) as Counts
    from Suppliers
    group by Country
    order by Counts DESC
""")

# TODO: Fetch your list of tuples and print it out
rows = cursor.fetchall()
print(rows)

[('USA', 4), ('Germany', 3), ('France', 3), ('UK', 2), ('Sweden', 2), ('Japan', 2), ('Italy', 2), ('Canada', 2), ('Australia', 2), ('Spain', 1), ('Singapore', 1), ('Norway', 1), ('Netherlands', 1), ('Finland', 1), ('Denmark', 1), ('Brazil', 1)]


In [25]:
# TODO: Convert your list of tuples to a dataframe
df = pd.DataFrame(rows, columns=['Country','Counts'])

# TODO: Generate a visualization on this dataframe
df

,Country,Counts
0,USA,4
1,Germany,3
2,France,3
3,UK,2
4,Sweden,2
5,Japan,2
6,Italy,2
7,Canada,2
8,Australia,2
9,Spain,1


### Question 3 : Most Frequent Country by Customer

What is the most common country of origin in our Customers table? Sort your output in descending order.

**Hint**: `GROUP BY` can be used to calculate grouped aggregates.

In [27]:
run_query('select * from Customers limit 3')

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021,Mexico
2,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,5023,Mexico


In [28]:
# TODO: Write a SQL query which will answer the question above.
cursor.execute("""
    select Country,count(Country) as CustomerCounts
    from Customers
    group by Country
    order by CustomerCounts DESC
""")

# TODO: Fetch your list of tuples and print it out
rows = cursor.fetchall()
print(rows)

[('USA', 13), ('Germany', 11), ('France', 11), ('Brazil', 9), ('UK', 7), ('Spain', 5), ('Mexico', 5), ('Venezuela', 4), ('Italy', 3), ('Canada', 3), ('Argentina', 3), ('Switzerland', 2), ('Sweden', 2), ('Portugal', 2), ('Finland', 2), ('Denmark', 2), ('Belgium', 2), ('Austria', 2), ('Poland', 1), ('Norway', 1), ('Ireland', 1)]


In [29]:
# TODO: Convert your list of tuples to a dataframe
df = pd.DataFrame(rows, columns=['Country','CustomerCounts'])

# TODO: Generate a visualization on this dataframe
df

,Country,CustomerCounts
0,USA,13
1,Germany,11
2,France,11
3,Brazil,9
4,UK,7
5,Spain,5
6,Mexico,5
7,Venezuela,4
8,Italy,3
9,Canada,3


### Question 4 : Least Popular Products by Quantity

What are the least popular products by order quantity? Limit your output to the bottom 20 products. Sort this table in ascending order and be sure to include the product name in your output and visualizations.

**Hint**: You will have to perform a join to calculate this answer.

In [ ]:
## least popular products by order - include product name
# limit 20
#sort asc

In [34]:
run_query(" select * from Products limit 3")

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18
1,2,Chang,1,1,24 - 12 oz bottles,19
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10


In [36]:
run_query("select * from Orderdetails limit 3")

,OrderDetailID,OrderID,ProductID,Quantity
0,1,10248,11,12
1,2,10248,42,10
2,3,10248,72,5


In [41]:
# TODO: Write a SQL query which will answer the question above.
cursor.execute ( """ 
     Select p.ProductName,o.Quantity
          from Products as p
          join Orderdetails as o
          on p.ProductID = o.ProductID
          order by o.Quantity ASC
          limit 20

""")

# TODO: Fetch your list of tuples and print it out
rows = cursor.fetchall()

In [42]:
# TODO: Convert your list of tuples to a dataframe
df = pd.DataFrame(rows, columns = ['ProductName','Quantity'])

# TODO: Generate a visualization on this dataframe
df

,ProductName,Quantity
0,Gravad lax,1
1,Teatime Chocolate Biscuits,1
2,Gudbrandsdalsost,1
3,Gnocchi di nonna Alice,2
4,Ravioli Angelo,2
5,Manjimup Dried Apples,2
6,Fløtemysost,2
7,Singaporean Hokkien Fried Mee,2
8,Boston Crab Meat,2
9,Alice Mutton,2


### Question 5 : Least Popular Products by Revenue

What are the least popular products by **total revenue** (order quantity * price)? Sort this table in ascending order and be sure to include the product name in your output and visualizations.

**Hint**: You will have to perform a join to calculate this answer.

In [ ]:
#least popular products by total revenue.- output product name
# revenue = quantity * price
# sort asc

In [47]:
# TODO: Write a SQL query which will answer the question above.
cursor.execute ( """ 
     Select p.ProductName,(o.Quantity * p.Price) as Revenue
          from Products as p
          join Orderdetails as o
          on p.ProductID = o.ProductID
          order by Revenue ASC
          

""")

# TODO: Fetch your list of tuples and print it out
rows = cursor.fetchall()

In [48]:
# TODO: Convert your list of tuples to a dataframe
df = pd.DataFrame(rows,columns=['Productname','Revenue'])

# TODO: Generate a visualization on this dataframe
df

,Productname,Revenue
0,Teatime Chocolate Biscuits,9.2
1,Konbu,12.0
2,Geitost,20.0
3,Spegesild,24.0
4,Gravad lax,26.0
...,...,...
513,Côte de Blaye,5270.0
514,Côte de Blaye,10540.0
515,Côte de Blaye,12911.5
516,Côte de Blaye,13175.0


### Question 6 : Countries with the Most Orders

Which country's have placed the most orders? For each country, list its' name in your output and visualizations.

**Hint**: You will have to perform a join to calculate this answer.

In [ ]:
# country- most ordered
# each counrty : output-country name
# join Customers and orderdetails

In [53]:
run_query("select * from Customers limit 3")

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021,Mexico
2,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,5023,Mexico


In [55]:
run_query("select * from Orderdetails limit 3")

,OrderDetailID,OrderID,ProductID,Quantity
0,1,10248,11,12
1,2,10248,42,10
2,3,10248,72,5


In [ ]:
# TODO: Write a SQL query which will answer the question above.

cursor.execute(""" 
    

""")
# TODO: Fetch your list of tuples and print it out
...

In [ ]:
# TODO: Convert your list of tuples to a dataframe
...

# TODO: Generate a visualization on this dataframe
...

### Question 7 : Countries with Customers Who Do Not Order

Which countries have at least one customer who has placed **no** orders? Count up the total number of customers who have placed no orders for each respective country. For each country, list its' name in your output and visualizations.

**Hint**: A join is necessary. We can also check if a column is `NULL` using the `IS` keyword.

In [ ]:
#Countries - at leat one customer with no order.
# 

In [ ]:
# TODO: Write a SQL query which will answer the question above.
...

# TODO: Fetch your list of tuples and print it out
...

In [ ]:
# TODO: Convert your list of tuples to a dataframe
...

# TODO: Generate a visualization on this dataframe
...

### Question 8 : Most Popular Suppliers

What are the most popular suppliers according to order count? List the supplier names and their number of orders. Sort your output in descending order.

**Hint**: You might need to use a subquery to join more than 2 tables together.

In [ ]:
#most popular supplier : order count
#supplier name,num of orders - 
# sort - desc

In [56]:
run_query("select * from Suppliers limit 3")

,SupplierID,SupplierName,ContactName,Address,City,PostalCode,Country,Phone
0,1,Exotic Liquid,Charlotte Cooper,49 Gilbert St.,Londona,EC1 4SD,UK,(171) 555-2222
1,2,New Orleans Cajun Delights,Shelley Burke,P.O. Box 78934,New Orleans,70117,USA,(100) 555-4822
2,3,Grandma Kelly's Homestead,Regina Murphy,707 Oxford Rd.,Ann Arbor,48104,USA,(313) 555-5735


In [57]:
run_query("select * from  Products limit 3")

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18
1,2,Chang,1,1,24 - 12 oz bottles,19
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10


In [58]:
run_query("select * from  Orderdetails limit 3")

,OrderDetailID,OrderID,ProductID,Quantity
0,1,10248,11,12
1,2,10248,42,10
2,3,10248,72,5


In [ ]:
# TODO: Write a SQL query which will answer the question above.
cursor.execute(" ")

# TODO: Fetch your list of tuples and print it out
...

In [ ]:
# TODO: Convert your list of tuples to a dataframe
...

# TODO: Generate a visualization on this dataframe
...

## Report

Answer the following analytical questions using numerical evidence.

### Question 1

Which countries have the greatest number of customers? How does this correlate with the number of suppliers by country (i.e. do more customers lead to less or more suppliers)? Which evidence supports your answer?

[Answer Here]

### Question 2

What is the least popular product by order quantity? How does this correlate with revenue (i.e. do less popular products by quantity lead to less or more revenue)? Which evidence supports your answer?

[Answer Here]

### Question 3

Which country has the most orders? How does this correlate with the number of customers who do not order (i.e. do countries with more ordering customers have more or less non-ordering customers)? Which evidence supports your answer?

[Answer Here]

### Question 4

Which supplier has the most orders? Which evidence supports your answer?

[Answer Here]